In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import hist.dask as dhist
import dask
import pickle
import os
import dask_awkward as dak

/tmp/ipykernel_2519566/3958481321.py:8: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  import coffea.nanoevents.methods.vector as vector


In [2]:
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")

In [15]:
with open('../filelists/q347_files.txt', 'r') as f:
#with open('../filelists/hbb_files.txt', 'r') as f:
    q347_files = [line.strip() for line in f]
qcd = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/qcd/300to470/' + q347_files[1]: "/Events"},
    #delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "bkg"},
).events()

In [4]:
with open('../filelists/hgg_files.txt', 'r') as f:
    hgg_files = [line.strip() for line in f]
hgg = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hgg/' + hgg_files[0]: "/Events"},
    #delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [35]:
genhiggs = hgg.GenPart[
    (hgg.GenPart.pdgId == 25)
    & hgg.GenPart.hasFlags(["fromHardProcess", "isLastCopy"])
]
parents = hgg.FatJet.nearest(genhiggs, threshold=0.2)
higgs_jets = ~ak.is_none(parents, axis=1)

cut = (
    (hgg.FatJet.pt > 200)
    & (hgg.FatJet.pt < 2500)
    & (hgg.FatJet.mass > 50)
    & (hgg.FatJet.mass < 200)
    & (abs(hgg.FatJet.eta) < 2.5)
    & (higgs_jets)  
)

In [36]:
hgg_fatjet = hgg.FatJet[cut]

In [37]:
pf = ak.flatten(hgg_fatjet.constituents.pf, axis=1)

In [38]:
jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.2)

In [39]:
cluster = fastjet.ClusterSequence(pf, jetdef)

In [40]:
#cluster.__dir__()

In [41]:
hgg_exclusive = cluster.exclusive_jets(n_jets=3).compute()

In [42]:
ak.all(ak.num(hgg_exclusive) == 3)

True

In [43]:
cut = (
    (qcd.FatJet.pt > 200)
    & (qcd.FatJet.pt < 2500)
    & (qcd.FatJet.mass > 50)
    & (qcd.FatJet.mass < 200)
    & (abs(qcd.FatJet.eta) < 2.5) 
)

In [44]:
fatjet = qcd.FatJet[cut]

In [45]:
pf = ak.flatten(fatjet.constituents.pf, axis=1)

In [46]:
jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.2)

In [47]:
cluster = fastjet.ClusterSequence(pf, jetdef)

In [48]:
qcd_exclusive = cluster.exclusive_jets(n_jets=3).compute()

In [49]:
ak.all(ak.num(qcd_exclusive) == 3)

True

In [34]:
def color_ring(fatjet, cluster_val):
    pf = ak.flatten(fatjet.constituents.pf, axis=1)
    jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, cluster_val)
    cluster = fastjet.ClusterSequence(pf, jetdef)
    subjets = cluster.exclusive_jets(n_jets=3)
    vec = ak.zip(
        {
            "x": subjets.px,
            "y": subjets.py,
            "z": subjets.pz,
            "t": subjets.E,
        },
        with_name="LorentzVector",
        behavior=vector.behavior,
    )
    vec = ak.pad_none(vec, 3)
    vec["norm3"] = np.sqrt(vec.dot(vec))
    vec["idx"] = ak.local_index(vec)
    i, j, k = ak.unzip(ak.combinations(vec, 3))
    #best = ak.argmin(abs((i + j + k).mass - 125), axis=1, keepdims=True)
    best = ak.argmax(abs((i + j + k).mass), axis=1, keepdims=True)
    order_check = ak.concatenate([i[best].mass, j[best].mass, k[best].mass], axis=1)
    largest = ak.argmax(order_check, axis=1, keepdims=True)
    smallest = ak.argmin(order_check, axis=1, keepdims=True)
    leading_particles = ak.concatenate([i[best], j[best], k[best]], axis=1)
    leg1 = leading_particles[largest]
    leg3 = leading_particles[smallest]
    leg2 = leading_particles[
        (leading_particles.idx != ak.flatten(leg1.idx))
        & (leading_particles.idx != ak.flatten(leg3.idx))
    ]
    leg1 = ak.firsts(leg1)
    leg2 = ak.firsts(leg2)
    leg3 = ak.firsts(leg3)
    a12 = np.arccos(leg1.dot(leg2) / (leg1.norm3 * leg2.norm3))
    a13 = np.arccos(leg1.dot(leg3) / (leg1.norm3 * leg3.norm3))
    a23 = np.arccos(leg2.dot(leg3) / (leg2.norm3 * leg3.norm3))
    color_ring = (a13**2 + a23**2) / (a12**2)
    return color_ring

In [56]:
uf_cr = ak.unflatten(
     color_ring(hgg_fatjet, cluster_val=0.2), counts=ak.num(hgg_fatjet)
).compute()

In [57]:
uf_cr

<Array [[2.28], [], [31.9], ..., [], [1.25], []] type='3000 * var * ?float64'>